In [ ]:
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
thar_std_red = fits.open('<Path to arc>/<arc_filename>')
thardata = np.genfromtxt('<Path to GHOSTDR>/GHOSTDR/ghostdr/ghost/lookups/Polyfit/ghost_thar_linelist_20220718.txt', usecols=(1,), dtype=None, encoding=None)

In [ ]:
import matplotlib

cmap = matplotlib.cm.get_cmap('viridis_r')
# cmap = matplotlib.cm.get_cmap('jet')
# cmap = matplotlib.cm.get_cmap('cividis')
norm = matplotlib.colors.Normalize(vmin=0, vmax=35) # inferno scale
# norm = matplotlib.colors.Normalize(vmin=4, vmax=10.5) # jet scale
# norm = matplotlib.colors.LogNorm(vmin=4, vmax=14)


plot_mcolors_b = [cmap(norm(order)) for order in np.arange(35)]

cmap = matplotlib.cm.get_cmap('inferno')
# cmap = matplotlib.cm.get_cmap('jet')
# cmap = matplotlib.cm.get_cmap('cividis')
norm = matplotlib.colors.Normalize(vmin=0, vmax=38) # inferno scale
# norm = matplotlib.colors.Normalize(vmin=4, vmax=10.5) # jet scale
# norm = matplotlib.colors.LogNorm(vmin=4, vmax=14)


plot_mcolors_r = [cmap(norm(order)) for order in np.arange(34)+5]
# print(len(plot_mcolors))


In [ ]:
wavemod = thar_std_red[4].data
wavemod

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)


# Set up the file that you are going to save to
pp = PdfPages('ThAr_red_lines_stdres.pdf')


fig = plt.figure(figsize = (10,15))


wavelengths = np.arange(5100, 10700, 20)
# wavelengths = np.arange(5200, 5500, 20)

# wavelengths = np.arange(3510, 3710, 20)


m_ref = 50
yprimes = np.arange(thar_std_red[1].data.shape[1])


for index, wavelength in enumerate(wavelengths):
    plot_id = (index % 5) + 1
    ax = fig.add_subplot(510 + plot_id)
    
    snrcut = 0.0
    ymax = 0.
    temp_max1 = 0.
    temp_max2 = 0.
    old_max = 0.
    
    for i, plotcolor in zip(range(thar_std_red[1].data.shape[0]), plot_mcolors_r):
#         print(i)
        m = i + 33
        poly_w_model = np.poly1d([np.poly1d(wavemod[i,:])((m_ref/m)-1) for i in range(len(wavemod[:,0]))])
        wave = poly_w_model(yprimes- 6144//2)
        data = thar_std_red[1].data[i,:,0]

        if len(data[(wave < wavelength + 10) & (wave > wavelength-10)]) > 0.5:
            old_max = temp_max1
            temp_max1 = np.max(data[(wave < wavelength + 10) & (wave > wavelength-10)])

        ax.plot(wave, data, c=plotcolor, lw=2)


            
    temp_max1 = np.maximum(old_max, temp_max1)


    old_max = 0.
    
    temp_max2 = np.maximum(old_max, temp_max2)

    
    present_thar_lines = thardata[(thardata < wavelength + 10) & (thardata > wavelength-10)]
#     print(present_cuar_lines)
    
    for wave in present_thar_lines:
        ax.plot(wave, -0.9*np.minimum(np.maximum(temp_max1, temp_max2),5000), marker = '|', ms = 20, mec='r', mew=1.5, ls = 'None', c='r')

    ax.set_ylabel(r'$\mathtt{Flux}$', fontsize=16)
    
    ax.xaxis.set_major_locator(MultipleLocator(2))
    ax.xaxis.set_minor_locator(MultipleLocator(0.5))


    ax.get_xaxis().get_major_formatter().set_useOffset(False)
    ax.get_xaxis().get_major_formatter().set_scientific(False)
    ax.set_xlim([wavelength -10, wavelength+10])


    ax.set_ylim([-1*np.minimum(np.maximum(temp_max1, temp_max2),5000)*1.2, np.minimum(np.maximum(temp_max1, temp_max2),5000)*1.2])

    if plot_id == 5 or index == len(wavelengths) - 1:
        ax.set_xlabel(r'$\mathtt{Wavelength \ (\AA)}$', fontsize=16)
        plt.tight_layout()
        pp.savefig()
        plt.close(fig)
        fig = plt.figure(figsize = (10,15))

# Once you've looped through everything, close the page
pp.close()